# [Module 1] Train a Keras Sequential Model (TensorFlow 2.1)

### [Note] 본 주피터 노트북은 TensorFlow 2.1에서 핸즈온을 수행합니다. Amazon SageMaker는 2020년 1월부터 빌트인 딥러닝 컨테이너 형태로 TensorFlow 2.0을 지원하고 있습니다.

본 노트북(notebook)은 SageMaker 상에서 Keras Sequential model을 학습하는 방법을 단계별로 설명합니다. 본 노트북에서 사용한 모델은 간단한 deep CNN(Convolutional Neural Network) 모델로 [the Keras examples](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py)에 소개된 모델과 동일합니다.
- 참고로, 본 모델은 25 epoch 학습 후에 검증셋의 정확도(accuracy)가 약 75%이고 50 epoch 학습 후에 검증셋의 정확도가 약 79% 입니다.
- 본 워크샵 과정에서는 시간 관계상 5 epoch까지만 학습합니다. (단, Horovod 기반 분산 학습은 10 epoch까지 학습합니다.)

In [1]:
import tensorflow as tf
import numpy as np
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

tf.__version__

Num GPUs Available:  0


'2.1.0'

In [1]:
!pip install -U sagemaker==1.72.1

     |████████████████████████████████| 299 kB 6.2 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-1.72.1-py2.py3-none-any.whl size=386572 sha256=9888701caaf5a69bb31207775310845085694f68e3f9a608f76fbfe87dbc0989
  Stored in directory: /home/ec2-user/.cache/pip/wheels/65/21/9e/e0cfa0f2891e29445c2e241cd9eb4d9dbee64a0a8554f1cf7f
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 0.1.5
    Uninstalling smdebug-rulesconfig-0.1.5:
      Successfully uninstalled smdebug-rulesconfig-0.1.5
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.16.4.dev0
    Uninstalling sagemaker-2.16.4.dev0:
      Successfully uninstalled sagemaker-2.16.4.dev0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p36/bin/python -m pip install --upgrade pip' command.


## Getting the data
아래 코드를 실행하여 Cifar dataset을 다운로드 받고 TFRecord데이터로 변환합니다. 
### TFRecord는 무엇인가요?
- Google에서 Tensorflow backend로 모델링 시에 공식적으로 권장하는 binary 포맷입니다.
- Tensorflow의 protocol buffer 파일로 직렬화된 입력 데이터가 담겨 있습니다.
- 대용량 데이터를 멀티스레딩으로 빠르게 스트리밍할 때 유용합니다. (모든 데이터는 메모리의 하나의 블록에 저장되므로, 입력 파일이 개별로 저장된 경우보다 데이터 로딩에 필요한 시간이 대폭 단축됩니다.)
- Example 객체로 구성된 배열의 집합체입니다. (an array of Examples)
- 아래 그림은 $m$차원 feautre가 $n$개의 샘플로 구성된 TFRecord 예시입니다.
![TFRecord](images/TFRecord.png)

In [2]:
#"generate_cifar10_tfrecords_v2.py"가 궁금할 경우 아래 #을 제거 하고 실행합니다.
#!cat generate_cifar10_tfrecords_v2.py

In [3]:
!python generate_cifar10_tfrecords_v2.py --data-dir ./data

Download from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and extract.
./data
170500096/170498071 [==============================] - 2s 0us/step
Generating ./data/train/train.tfrecords
Generating ./data/validation/validation.tfrecords
Generating ./data/eval/eval.tfrecords
Done!


## Run the training locally

 스크립트는 모델 학습에 필요한 인자값(arguments)들을 사용합니다. 모델 학습에 필요한 인자값들은 아래와 같습니다.

1. `model_dir` - 로그와 체크 포인트를 저장하는 경로
2. `train, validation, eval` - TFRecord 데이터셋을 저장하는 경로
3. `epochs` - epoch 횟수

아래 명령어로 **<font color='red'>SageMaker 관련 API 호출 없이</font>** 로컬 노트북 인스턴스 환경에서 1 epoch만 학습해 봅니다. 참고로, MacBook Pro(15-inch, 2018) 2.6GHz Core i7 16GB 사양에서 2분 20초~2분 40초 소요됩니다.

In [19]:
%%time
!mkdir -p logs
!python training_script/cifar10_keras_tf2.py --model_dir ./logs \
                                         --train data/train \
                                         --validation data/validation \
                                         --eval data/eval \
                                         --epochs 1
!rm -rf logs

2020-12-07 14:43:45.093545: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2020-12-07 14:43:45.116155: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2999990000 Hz
2020-12-07 14:43:45.116583: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55cb724a48e0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-12-07 14:43:45.116606: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-12-07 14:43:45.116698: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Train for 312 steps, validate for 78 steps
312/312 [==============================] - 35s 113ms/step - loss: 1.8524 - accuracy: 0.3207 - val_loss: 1.8120 - val_accuracy: 0.3612
2020-12-07

## Use TensorFlow Script Mode

TensorFlow 버전 1.11 이상에서 Amazon SageMaker Python SDK는 **스크립트 모드(Script mode)**를 지원합니다. 스크립트 모드는 종래 레거시 모드(Legacy mode) 대비 아래 장점들이 있습니다.

* 스크립트 모드의 학습 스크립트는 일반적으로 TensorFlow 용으로 작성하는 학습 스크립트와 더 유사하므로 TensorFlow 학습 스크립트를 최소한의 변경으로 실행할 수 있습니다. 따라서, 기존 레거시 모드보다 TensorFlow 학습 스크립트를 수정하는 것이 더 쉽습니다. 
    - 레거시 모드는 Tensorflow Estimator API를 기반으로 한 아래의 함수들을 반드시 포함해야 합니다.
        - 아래 함수들에서 하나의 함수를 만드시 포함해야 합니다.
            - `model_fn`: 학습할 모델을 정의합니다,
            - `keras_model_fn`: 학습할 tf.keras 모델을 정의합니다.
            - `estimator_fn`: 학습할 tf.estimator.Estimator를 정의합니다.
        - `train_input_fn`: 학습 데이터 로딩과 전처리를 수행합니다. 
        - `eval_input_fn`: 검증 데이터의 로딩과 전처리를 수행합니다.
        - (Optional) `serving_input_fn`: 예측(prediction) 중에 모델에 전달할 feautre를 정의합니다. 이 함수는 학습시에만 사용되지만, SageMaker 엔드포인트에서 모델을 배포할 때 필요합니다.
    - `if __name__ == “__main__”:` 블록을 정의할 수 없어 디버깅이 쉽지 않습니다.
    
* 스크립트 모드는 Python 2.7-와 Python 3.6-을 지원합니다.

* 스크립트 모드는 **Hovorod 기반 분산 학습(distributed training)도 지원**합니다.

TensorFlow 스크립트 모드에서 학습 스크립트를 작성하는 방법 및 Tensorflow 스크립트 모드의 estimator와 model 사용법에 대한 자세한 내용은
https://sagemaker.readthedocs.io/en/stable/using_tf.html 을 참조하세요.

### Preparing your script for training in SageMaker

SageMaker 스크립트 모드의 학습 스크립트는 SageMaker 외부에서 실행할 수 있는 학습 스크립트와 매우 유사합니다.
SageMaker는 하나의 인자값(argument), model_dir와 로그 및 모델 아티팩트(model artifacts)에 사용되는 S3 경로로 학습 스크립트를 실행합니다.

SageMaker 학습 인스턴스에서는 학습의 컨테이너에 S3에 저장된 데이터를 다운로드하여 학습에 활용합니다. 그 때, S3 버킷의 데이터 경로와 컨테이너의 데이터 경로를 컨테이너 환경 변수를 통해 연결합니다.

여러분은 다양한 환경 변수를 통해 학습 환경에 대한 유용한 속성들(properties)에 액세스할 수 있습니다.
이 스크립트의 경우 `Train, Validation, Eval`이라는 3 개의 데이터 채널을 스크립트로 보냅니다.

----
### TODO 1.
`cifar10_keras_sm_tf2.py`파일에서 SageMaker API 환경 변수 SM_CHANNEL_TRAIN, SM_CHANNEL_VALIDATION, SM_CHANNEL_EVAL에서 디폴트 값을 가져오기 위해 train, validation, eval 인수를 수정해 주세요. 

`cifar10_keras_sm_tf2.py`의 `if __name__ == '__main__':` 블록 내에 아래 인자값을 수정해 주세요.

```python
parser.add_argument(
        '--train',
        type=str,
        required=False,
        default=os.environ.get('SM_CHANNEL_TRAIN'), # <-- 수정 부분
        help='The directory where the CIFAR-10 input data is stored.')
parser.add_argument(
        '--validation',
        type=str,
        required=False,
        default=os.environ.get('SM_CHANNEL_VALIDATION'), # <-- 수정 부분
        help='The directory where the CIFAR-10 input data is stored.')
parser.add_argument(
        '--eval',
        type=str,
        required=False,
        default=os.environ.get('SM_CHANNEL_EVAL'), # <-- 수정 부분
        help='The directory where the CIFAR-10 input data is stored.')
```


환경 변수에 따른 S3 경로와 컨테이너 경로는 아래 표와 같습니다.

|  S3 경로  |  환경 변수  |  컨테이너 경로  |
| :---- | :---- | :----| 
|  s3://bucket_name/prefix/train  |  `SM_CHANNEL_TRAIN`  | `/opt/ml/input/data/train`  |
|  s3://bucket_name/prefix/validation  |  `SM_CHANNEL_VALIDATION`  | `/opt/ml/input/data/validation`  |
|  s3://bucket_name/prefix/eval  |  `SM_CHANNEL_EVAL`  | `/opt/ml/input/data/eval`  |
|  s3://bucket_name/prefix/model.tar.gz  |  `SM_MODEL_DIR`  |  `/opt/ml/model`  |
|  s3://bucket_name/prefix/output.tar.gz  |  `SM_OUTPUT_DATA_DIR`  |  `/opt/ml/output/data`  |

얘를 들어, `/opt/ml/input/data/train`은 학습 데이터가 다운로드되는 컨테이너 내부의 디렉토리입니다.

자세한 내용은 아래의 SageMaker Python SDK 문서를 확인하시기 바랍니다.<br>
(https://sagemaker.readthedocs.io/en/stable/using_tf.html#preparing-a-script-mode-training-script)


SageMaker는 train, validation, eval 경로들을 직접 인자로 보내지 않고, 대신 스크립트에서 환경 변수를 사용하여 해당 인자를 필요하지 않은 것으로 표시합니다.

SageMaker는 유용한 환경 변수를 여러분이 작성한 학습 스크립트로 보냅니다. 예시들은 아래와 같습니다.
* `SM_MODEL_DIR`: 학습 작업이 모델 아티팩트(model artifacts)를 저장할 수 있는 로컬 경로를 나타내는 문자열입니다. 학습 완료 후, 해당 경로 내 모델 아티팩트는 모델 호스팅을 위해 S3에 업로드됩니다. 이는 S3 위치인 학습 스크립트에 전달 된 model_dir 인수와 다르다는 점을 주의해 주세요. SM_MODEL_DIR은 항상 `/opt/ml/model`로 설정됩니다.
* `SM_NUM_GPUS`: 호스트(Host)에서 사용 가능한 GPU 수를 나타내는 정수(integer)입니다.
* `SM_OUTPUT_DATA_DIR`: 출력 아티팩트를 저장할 디렉토리의 경로를 나타내는 문자열입니다. 출력 아티팩트에는 체크포인트, 그래프 및 다른 저장용 파일들이 포함될 수 있지만 모델 아티팩트는 포함되지 않습니다. 이 출력 아티팩트들은 압축되어 모델 아티팩트와 동일한 접두사가 있는 S3 버킷으로 S3에 업로드됩니다.

이 샘플 코드는 네트워크 지연을 줄이기 위해 모델의 체크포인트(checkpoints)를 로컬 환경에 저장합니다. 이들은 학습 종료 후 S3에 업로드할 수 있습니다.

----
### TODO 2.

`cifar10_keras_sm_tf2.py`의 `if __name__ == '__main__':` 블록 내에 아래 인자값을 추가해 주세요.

```python
parser.add_argument(
        '--model_output_dir',
        type=str,
        default=os.environ.get('SM_MODEL_DIR'))
```

----
### TODO 3.
`ModelCheckpoint` 함수의 저장 경로를 새 경로로 아래와 같이 수정해 주세요.

From:
```python
callbacks.append(ModelCheckpoint(args.model_dir + '/checkpoint-{epoch}.h5'))
```
To:
```python
callbacks.append(ModelCheckpoint(args.model_output_dir + '/checkpoint-{epoch}.h5'))
```

----
### TODO 4.
`save_model` 함수의 인자값을 아래와 같이 수정해 주세요.

From:  
```python
return save_model(model, args.model_dir)
```
To:  
```python
return save_model(model, args.model_output_dir)
```


### Test your script locally (just like on your laptop)

테스트를 위해 위와 동일한 명령(command)으로 새 스크립트를 실행하고, 예상대로 실행되는지 확인합니다. <br>
SageMaker TensorFlow API 호출 시에 환경 변수들은 자동으로 넘겨기지만, 로컬 주피터 노트북에서 테스트 시에는 수동으로 환경 변수들을 지정해야 합니다. (아래 예제 코드를 참조해 주세요.)

```python
%env SM_MODEL_DIR=./logs
```

In [20]:
%%time
!mkdir -p logs   

# Number of GPUs on this machine
%env SM_NUM_GPUS=1
# Where to save the model
%env SM_MODEL_DIR=./logs
# Where the training data is
%env SM_CHANNEL_TRAIN=data/train
# Where the validation data is
%env SM_CHANNEL_VALIDATION=data/validation
# Where the evaluation data is
%env SM_CHANNEL_EVAL=data/eval


!python training_script/cifar10_keras_sm_tf2.py --model_dir ./logs --epochs 1
!rm -rf logs

env: SM_NUM_GPUS=1
env: SM_MODEL_DIR=./logs
env: SM_CHANNEL_TRAIN=data/train
env: SM_CHANNEL_VALIDATION=data/validation
env: SM_CHANNEL_EVAL=data/eval
2020-12-07 14:44:28.970600: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2020-12-07 14:44:28.992151: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2999990000 Hz
2020-12-07 14:44:28.992573: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ed1d22db40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-12-07 14:44:28.992595: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-12-07 14:44:28.992684: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Train for 312 steps, validate for 78

### Use SageMaker local for local testing

본격적으로 학습을 시작하기 전에 로컬 모드를 사용하여 디버깅을 먼저 수행합니다. 로컬 모드는 학습 인스턴스를 생성하는 과정이 없이 로컬 인스턴스로 컨테이너를 가져온 후 곧바로 학습을 수행하기 때문에 코드를 보다 신속히 검증할 수 있습니다.

Amazon SageMaker Python SDK의 로컬 모드는 TensorFlow 또는 MXNet estimator서 단일 인자값을 변경하여 CPU (단일 및 다중 인스턴스) 및 GPU (단일 인스턴스) SageMaker 학습 작업을 에뮬레이션(enumlate)할 수 있습니다. 

로컬 모드 학습을 위해서는 docker-compose 또는 nvidia-docker-compose (GPU 인스턴스인 경우)의 설치가 필요합니다. 아래 코드 셀을 통해 본 노트북 환경에 docker-compose 또는 nvidia-docker-compose를 설치하고 구성합니다. 
 
로컬 모드의 학습을 통해 여러분의 코드가 현재 사용 중인 하드웨어를 적절히 활용하고 있는지 확인하기 위한 GPU 점유와 같은 지표(metric)를 쉽게 모니터링할 수 있습니다.

In [21]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [22]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='2.1.0',
                       py_version='py3',
                       script_mode=True,
                       hyperparameters={'epochs' : 1},
                       train_instance_count=1, 
                       train_instance_type='local')

학습을 수행할 3개의 채널과 데이터의 경로를 지정합니다. **로컬 모드로 수행하기 때문에 S3 경로 대신 노트북 인스턴스의 경로를 지정하시면 됩니다.**

In [23]:
%%time
estimator.fit({'train': 'file://data/train',
               'validation': 'file://data/validation',
               'eval': 'file://data/eval'})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Creating tmpq_52v6n8_algo-1-5nxcr_1 ... 
Attaching to tmpq_52v6n8_algo-1-5nxcr_12mdone
algo-1-5nxcr_1  | 2020-12-07 14:45:28,501 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-5nxcr_1  | 2020-12-07 14:45:28,507 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-5nxcr_1  | 2020-12-07 14:45:29,696 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-5nxcr_1  | 2020-12-07 14:45:29,710 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-5nxcr_1  | 2020-12-07 14:45:29,722 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-5nxcr_1  | 2020-12-07 14:45:29,731 sagemaker-containers INFO     Invoking user script
algo-1-5nxcr_1  | 
algo-1-5nxcr_1  | Training Env:
algo-1-5nxcr_1  | 
algo-1-5nxcr_1  | {
algo-1-5nxcr_1  |     "additional_framework_parameters": {},
algo-1-5nxcr_1  |     "channel_input_dirs": {
algo-1-5nxcr_1  

algo-1-5nxcr_1  | WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
algo-1-5nxcr_1  | Instructions for updating:
algo-1-5nxcr_1  | If using Keras pass *_constraint arguments to layers.
algo-1-5nxcr_1  | WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
algo-1-5nxcr_1  | Instructions for updating:
algo-1-5nxcr_1  | If using Keras pass *_constraint arguments to layers.
algo-1-5nxcr_1  | INFO:tensorflow:Assets written to: /opt/ml/model/1/assets
algo-1-5nxcr_1  | INFO:tensorflow:Assets written to: /opt/ml/model/1/assets
algo-1-5nxcr_1  

Estimator가 처음 실행될 때 Amazon ECR 리포지토리(repository)에서 컨테이너 이미지를 다운로드해야 하지만 학습을 즉시 시작할 수 있습니다. 즉, 별도의 학습 클러스터가 프로비저닝 될 때까지 기다릴 필요가 없습니다. 또한 반복 및 테스트시 필요할 수 있는 후속 실행에서 MXNet 또는 TensorFlow 스크립트에 대한 수정 사항이 즉시 실행되기 시작합니

### Dependency 설정 및 Laucher code 작성하기

* Container에 없는 파일을 추가적으로 설치하거나 dependency code를 패스 하기 귀해 다음과 launcher code를 사용할 수 있습니다.
  이 예제에서는 ```" cifar10_keras_sm_tf2_dependency.py"``` 에 ```import jsonlines```를 추가하였고 프로그램 시작 전에 requirment.txt에 페키지를 pip로 설치하도록 합니다.  
* Requirements.txt를 위의 Python 실행 코드와 함께 동일 폴더에 저장합니다. 
* Script모드에서 AWS Deeplearning container에 requirements.txt 에 정의된 패키를 설치한 후 실행하게 됩니다.
* 참고: https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html?highlight=requirements#id14 

In [24]:
data_dir = os.path.join(os.getcwd(), 'training_script')
os.makedirs(data_dir, exist_ok=True)
data_dir

'/home/ec2-user/SageMaker/startup-sagemaker-workshop/training_script'

In [25]:
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/local_mode_setup.sh
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/daemon.json    
!/bin/bash ./local_mode_setup.sh

SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


In [26]:
%%writefile training_script/launcher.sh

/usr/bin/python3 -m pip install --upgrade pip
pip install -r requirements.txt


python cifar10_keras_sm_tf2_dependency.py --model_dir ${SM_MODEL_DIR} --epochs ${SM_HP_EPOCHS}

echo "Generated image $(ls ${SM_MODEL_DIR})"



Overwriting training_script/launcher.sh


In [27]:
!cat training_script/launcher.sh


/usr/bin/python3 -m pip install --upgrade pip
pip install -r requirements.txt


python cifar10_keras_sm_tf2_dependency.py --model_dir ${SM_MODEL_DIR} --epochs ${SM_HP_EPOCHS}

echo "Generated image $(ls ${SM_MODEL_DIR})"



In [28]:
train_instance_type='local'

이제 requirement.txt가 시작 전에 설치 되고 시작되었는지 확인해 봅니다. 예제에서는 requiremet.txt에 dummy로 jsonlines를 import하고 Jsonline object에서 1,2,3을 프린트 하도록 해 놓았습니다. 

In [29]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='launcher.sh',
                       dependencies=['training_script/cifar10_keras_sm_tf2_dependency.py'],
                       source_dir=data_dir,
                       train_instance_type='local',
                       train_instance_count=1,
                       role=role,
                       hyperparameters={'epochs' : 1},
                       framework_version='2.1.0',
                       py_version='py3',
                       script_mode=True)


In [30]:
%%time
estimator.fit({'train': 'file://data/train',
               'validation': 'file://data/validation',
               'eval': 'file://data/eval'
               })

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Creating tmpyaaafza6_algo-1-4bra8_1 ... 
Attaching to tmpyaaafza6_algo-1-4bra8_12mdone
algo-1-4bra8_1  | 2020-12-07 14:46:34,797 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-4bra8_1  | 2020-12-07 14:46:34,804 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-4bra8_1  | 2020-12-07 14:46:35,977 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-4bra8_1  | 2020-12-07 14:46:35,991 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-4bra8_1  | 2020-12-07 14:46:36,005 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-4bra8_1  | 2020-12-07 14:46:36,015 sagemaker-containers INFO     Invoking user script
algo-1-4bra8_1  | 
algo-1-4bra8_1  | Training Env:
algo-1-4bra8_1  | 
algo-1-4bra8_1  | {
algo-1-4bra8_1  |     "additional_framework_parameters": {},
algo-1-4bra8_1  |     "channel_input_dirs": {
algo-1-4bra8_1  

algo-1-4bra8_1  | Successfully installed pip-20.3.1
algo-1-4bra8_1  | Collecting jsonlines
algo-1-4bra8_1  |   Downloading jsonlines-1.2.0-py2.py3-none-any.whl (7.6 kB)
algo-1-4bra8_1  | Requirement already satisfied: six in /usr/local/lib/python3.6/dist-packages (from jsonlines->-r requirements.txt (line 1)) (1.14.0)
algo-1-4bra8_1  | Installing collected packages: jsonlines
algo-1-4bra8_1  | Successfully installed jsonlines-1.2.0
algo-1-4bra8_1  | -----------requirment.txt 테스트 start----------------
algo-1-4bra8_1  | 1 
algo-1-4bra8_1  | 
algo-1-4bra8_1  | 2 
algo-1-4bra8_1  | 
algo-1-4bra8_1  | 3 
algo-1-4bra8_1  | 
algo-1-4bra8_1  | ------------requirment.txt 테스트 end -----------------
algo-1-4bra8_1  | Train for 312 steps, validate for 78 steps
312/312 [==============================] - 34s 110ms/step - loss: 1.8173 - accuracy: 0.3292 - val_loss: 2.1864 - val_accuracy: 0.2646
algo-1-4bra8_1  | 2020-12-07 14:47:19.696343: W tensorflow/python/util/util.cc:319] Sets are not currently c

### Using SageMaker for faster training time

이번에는 로컬 모드를 사용하지 않고 SageMaker 학습에 GPU 학습 인스턴스를 생성하여 학습 시간을 단축해 봅니다.<br>
로컬 모드와 다른 점들은 (1) `train_instance_type`이 로컬 모드의 ‘local’ 대신 여러분이 원하는 특정 인스턴스 유형으로 설정해야 하고, (2) 학습 데이터를 Amazon S3에 업로드 후 학습 경로를 S3 경로로 설정해야 합니다. 

SageMaker SDK는 S3 업로드를 위한 간단한 함수(`Session.upload_data()`)를 제공합니다. 이 함수를 통해 리턴되는 값은 데이터가 저장된 S3 경로입니다.
좀 더 자세한 설정이 필요하다면 SageMaker SDK 대신 boto3를 사용하시면 됩니다.

*[Note]: 고성능 워크로드를 위해 Amazon EFS와 Amazon FSx for Lustre도 지원하고 있습니다. 자세한 정보는 아래의 AWS 블로그를 참조해 주세요.<br>
https://aws.amazon.com/blogs/machine-learning/speed-up-training-on-amazon-sagemaker-using-amazon-efs-or-amazon-fsx-for-lustre-file-systems/*

In [31]:
dataset_location = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-cifar10')
display(dataset_location)

's3://sagemaker-us-east-2-870180618679/data/DEMO-cifar10'

S3에 데이터 업로드를 완료했다면, Estimator를 새로 생성합니다. <br>
아래 코드를 그대로 복사 후에 `train_instance_type='local'`을 `train_instance_type='ml.p2.xlarge'`로 수정하고
`hyperparameters={'epochs': 1}`를 `hyperparameters={'epochs': 5}`로 수정합니다.

```python
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
                       script_mode=True,                       
                       hyperparameters={'epochs': 1},
                       train_instance_count=1, 
                       train_instance_type='local')
```

*[Note] 
2019년 8월부터 SageMaker에서도 학습 인스턴스에 EC2 spot instance를 사용하여 비용을 크게 절감할 수 있습니다. 자세한 정보는 아래의 AWS 블로그를 참조해 주세요.<br>
https://aws.amazon.com/ko/blogs/korea/managed-spot-training-save-up-to-90-on-your-amazon-sagemaker-training-jobs/*

만약 Managed Spot Instance로 학습하려면 다음 코드를 Estimator의 train_instance_type의 다음 행에 추가해 주세요.
```python
train_max_run = 3600,
train_use_spot_instances = 'True',
train_max_wait = 3600,
```

In [32]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir=data_dir,
                       role=role,
                       framework_version='2.1.0',
                       py_version='py3',
                       script_mode=True,                       
                       hyperparameters={'epochs': 5},
                       train_instance_count=1, 
                       train_instance_type='ml.p3.2xlarge',
                       train_max_run = 3600,
                       train_use_spot_instances = 'True',
                       train_max_wait = 3600,
                       )

학습을 수행합니다. 이번에는 각각의 채널(`train, validation, eval`)에 S3의 데이터 저장 위치를 지정합니다.<br>
학습 완료 후 Billable seconds도 확인해 보세요. Billable seconds는 실제로 학습 수행 시 과금되는 시간입니다.
```
Billable seconds: <time>
```

참고로, `ml.p2.xlarge` 인스턴스로 5 epoch 학습 시 전체 6분-7분이 소요되고, 실제 학습에 소요되는 시간은 3분-4분이 소요됩니다.

In [33]:
%%time
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-12-07 14:47:27 Starting - Starting the training job...
2020-12-07 14:47:29 Starting - Launching requested ML instances......
2020-12-07 14:48:35 Starting - Preparing the instances for training......
2020-12-07 14:49:35 Downloading - Downloading input data...
2020-12-07 14:50:10 Training - Downloading the training image...
2020-12-07 14:50:44 Training - Training image download completed. Training in progress..2020-12-07 14:50:49,332 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-12-07 14:50:49,800 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparame

#015  1/312 [..............................] - ETA: 4s - loss: 1.4304 - accuracy: 0.3984#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  6/312 [..............................] - ETA: 3s - loss: 1.5394 - accuracy: 0.4258#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 11/312 [>.............................] - ETA: 3s - loss: 1.5136 - accuracy: 0.4311#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

#015  1/312 [..............................] - ETA: 4s - loss: 1.2018 - accuracy: 0.5938#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  7/312 [..............................] - ETA: 3s - loss: 1.2342 - accuracy: 0.5748#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 12/312 [>.............................] - ETA: 3s - loss: 1.2065 - accuracy: 0.5749#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

INFO:tensorflow:Assets written to: /opt/ml/model/1/assets
INFO:tensorflow:Assets written to: /opt/ml/model/1/assets
[2020-12-07 14:51:39.134 ip-10-0-133-68.us-east-2.compute.internal:27 INFO utils.py:25] The end of training job file will not be written for jobs running under SageMaker.
2020-12-07 14:51:40,203 sagemaker-containers INFO     Reporting training SUCCESS

2020-12-07 14:51:53 Uploading - Uploading generated training model
2020-12-07 14:51:53 Completed - Training job completed
Training seconds: 138
Billable seconds: 41
Managed Spot Training savings: 70.3%
CPU times: user 544 ms, sys: 17.3 ms, total: 561 ms
Wall time: 4min 43s


수고하셨습니다.

SageMaker에서 GPU 인스턴스를 사용해 5 epoch를 정상적으로 학습할 수 있었습니다.
다음 노트북으로 계속 진행하기 전에 SageMaker 콘솔의 Training jobs 섹션을 살펴보고 여러분이 수행한 job을 찾아 configuration을 확인하세요.

스크립트 모드 학습에 대한 자세한 내용은 아래의 AWS 블로그를 참조해 주세요.
Using TensorFlow eager execution with Amazon SageMaker script mode
